#  The Shared Library with GCC

**C/C++ Under Linux**

* %cd Change the current working directory to `./demo`

In [4]:
%cd demo

[WinError 2] 系统找不到指定的文件。: 'demo'
F:\SEU\SEE\PySEE\home\notebook\demo


In [2]:
%pwd

'F:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

When your program is linked against a shared library, only a small table is created in the executable. Before the executable starts running, **the operating system loads the machine code needed for the external functions** - a process known as **dynamic linking.** 

   
 ![dynamic](./img/dynamic-lib.png)  

    
* Dynamic linking makes executable files smaller and saves disk space, because `one` copy of a **library** can be **shared** between `multiple` programs. 


* Furthermore, most operating systems allows one copy of a shared library in memory to be used by all running programs, thus, saving memory. 


* The shared library codes can be upgraded without the need to recompile your program.


A **shared library** has file extension of 

   * **`.so`** (shared objects) in `Linux(Unixes)`
   
   
   * **`.dll** (dynamic link library) in `Windows`. 
   



## 1: Building the shared library

The shared library we will build consist of a single source file: 

* `statistics.c/h`

We will compile the C file with `Position Independent Code( PIC )` into a shared library。

GCC assumes that all libraries 
   
* `start` with `lib`

* `end`  with `.dll`(windows) or `.so`(Linux)，

so, we should name the  shared library begin with `lib prefix` and the `.so/.dll` extensions.

* libstat.dll(Windows)

* libstat.so(Linux)


### 1.1 Building DLL for  Windows

In [5]:
!gcc -c -O3 -Wall -fPIC -o ./obj/statistics.o  ./src/statistics.c -I./include/
!gcc -shared -o ./bin/libstat.dll  ./obj/statistics.o

**Makefile of Building DLL for Windows**

In [6]:
%%file ./makefile-statdll.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC 

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: libdll

libdll: obj
	 $(CC) -shared -o $(BINDIR)libstat.dll $(OBJDIR)statistics.o
    
obj: $(SRCDIR)statistics.c
	 $(CC) -c $(CFLAGS)  -o $(OBJDIR)statistics.o $(SRCDIR)statistics.c -I$(INCDIR)
     


Writing ./makefile-statdll.mk


In [7]:
!make -f makefile-statdll.mk

gcc -shared -o ./bin/libstat.dll ./obj/statistics.o


In [8]:
!dir .\bin\libstat.dll

 Volume in drive F is cmh
 Volume Serial Number is 9C25-3306

 Directory of F:\SEU\SEE\PySEE\home\notebook\demo\bin

2020/03/12  12:00           126,742 libstat.dll
               1 File(s)        126,742 bytes
               0 Dir(s)  98,837,856,256 bytes free



* `-c`: compile into object file with default name : funs.o.

      By default, the object file has the same name as the source file with extension of ".o" 
  
  
* `-O3`: Optimize yet more.

      turns on all optimizations specified by -O2 and also turns on the -finline-functions, -fweb, -frename-registers and -funswitch-loops optionsturns  on  all  optimizations   
     
  
* `-Wall`: prints "all"  compiler's warning message. 

      This option should always be used, in order to generate better code.


* **`-fPIC`** : stands for `Position Independent Code`(位置无关代码)
   
   the generated machine code is `not dependent` on being located at a `specific address` in order to `work`.
   
   Position-independent code can be `executed` at `any memory address`
    
     
* **-shared:** creating a shared library


### 1.2 Building SO under  Linux

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./obj/statistics.o ./gcc/statistics.c -I$(INCDIR)
!gcc -shared -o ./bin/libstat  ./obj/statistics.o 

In [ ]:
!ls ./bin/libstat.*

#### Linux 

#####  ls

Linux Coommand: list files
```bash
$ls
```

##### Path separator

* Path separators

   * Linux: `/`

   * Windows: **\**

* Eescape character `\`

   * The backslash`\` is an `escape character`,you use to inform that the next character is special.
 

**Makefile of Building SO for Linux**

In [9]:
%%file .makefile-statso.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: libdll

libdll: obj
	 $(CC) -shared -o $(BINDIR)libstat $(OBJDIR)statistics.o
   
obj: (SRCDIR)statistics.c
	 $(CC) -c $(CFLAGS)  -o $(OBJDIR)statistics.o $(SRCDIR)statistics.c -I$(INCDIR)


Writing .makefile-statso.mk


In [ ]:
!make -f makefile-statso.mk

In [ ]:
!ls ./bin/libstat.so

## 2 Building a Client Executable 

### 2.1 Header Files and Libraries 

* `Header File`: When compiling the program, the **compiler** needs the **header** files to compile the source codes;

* `libraries`: the **linker** needs the **libraries** to resolve external references from other object files or libraries. 

The `compiler` and `linker` will not find the `headers/libraries` unless you set **the appropriate options**

* **1 Searching for Header Files**

   **`-Idir`:** The include-paths are specified via **-Idir** option (`uppercase` 'I' followed by the directory path or environment variable **CPATH**). 
   
   
* **2 Searching for libraries Files**

   **`-Ldir`**: The library-path is specified via **-Ldir** option (`uppercase` 'L' followed by the directory path(or environment variable **LIBRARY_PATH**). 


* **3 Linking the library**

   **`-llibname`**: Link with the library name **without** the `lib` prefix and the `.so/.dll` extensions.
  
        Windows
    ```bash
         -I./include/ -L./bin/ -lstat
    ```
        Linux
    ```bash
         -I./include/ -L./bin/ -lstat -Wl,-rpath=./bin/  
    ```

   * **`-Wl,option`**

    Pass option as an option to the **linker**. If option contains `commas`, it is split into multiple options at the commas. You can use this syntax to pass an argument to the option. For example, -Wl,-Map,output.map passes -Map output.map to the linker. When using the GNU linker, you can also get the same effect with `-Wl,-Map=output.map'.
    
   * **`-rpath=dir`** 

    **Add a directory to the runtime library search path**. This is used when linking an ELF executable with shared objects. All -rpath arguments are concatenated and passed to the runtime linker, which uses them to locate shared objects at runtime. The -rpath option is also used when locating shared objects which are needed by shared objects explicitly included in the link;
   


### 2.2 Calling the Shared Library

The following source code `"mainSum.c"` demonstrates calling the DLL's functions: 

**NOTE:** `mainSum.c` is the **same** code in multi-source example

In [ ]:
# %load ./src/statApp.c

#include <stdio.h> 
#include "statistics.h"

int main() {
     double b[] = {8, 4, 5, 3, 2};
     printf("mean is %f\n", mean(b,5)); 
     return 0;
}


#### 2.2.1 Windows

In [11]:
!gcc -c -o ./obj/statApp.o ./src/statApp.c -I./include/
!gcc -o  ./bin/statApp ./obj/statApp.o  -L./bin/ -lstat

In [12]:
!.\bin\statApp

mean is 4.400000


**Makefile under Windows**

In [17]:
%%file ./makefile-statapp-dll.mk

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: mainexe

mainexe: sumobj  
	gcc -o $(BINDIR)statApp $(OBJDIR))statApp.o -L$(BINDIR) -lstat

sumobj: $(SRCDIR))statApp.c 
	gcc -c -o $(OBJDIR)statApp.o $(SRCDIR)statApp.c -I$(INCDIR)

Overwriting ./makefile-statapp-dll.mk


In [16]:
!make -f makefile-statapp-dll.mk

make: *** No rule to make target 'src/)statApp.c', needed by 'sumobj'.  Stop.


In [18]:
!.\bin\statAPp

mean is 4.400000


#### 2.2.2 Linux

In [ ]:
!gcc -c -o ./obj/statApp.o ./src/statApp.c -I./include/ 
!gcc -o  ./bin/statApp ./obj/statApp.o  -L./bin/ -lstat -Wl,-rpath=./bin/ 

**makefile under Linux**

In [19]:
%%file ./makefile-statapp-so.mk

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: statApp

statApp: statobj 
	gcc -o $(BINDIR)statApp $(OBJDIR)statApp.o -L$(BINDIR) -lstat -Wl,-rpath=./bin/ 

statobj: $(SRCDIR)statApp.c 
	gcc -c -o $(OBJDIR)statApp.o $(SRCDIR)statApp.c -I$(INCDIR)  
     

Writing ./makefile-statapp-so.mk


In [ ]:
!make -f makefile-statapp-so.mk

In [ ]:
!./bin/statApp

## 3 Building  a Shared Library with Multi-source

### 3.1  The multi-source files
The shared library we will build consist of the multi-source files

* funs.c/h

* statistics.c/h

In [23]:
%%file ./include/funs.h

#ifndef FUNS_H
#define FUNS_H

int factorial(int n);

#endif

Writing ./include/funs.h


In [24]:
%%file ./src/funs.c

#include "funs.h"

// The factorial of a positive integer n, denoted by n!, is the product of all positive integers less than or equal to n. 
//  For example,5!=5*4*3*2*1=120
//  The value of 0! is 1 
int factorial(int n)
{
    if (n == 0 ) {
        return 1;
    }
    else 
    {
        return n * factorial(n - 1);
    }
}

Overwriting ./src/funs.c


### 3.2 Building  into One Shared Library

#### 3.2.1 Windows's DLL

In [20]:
!gcc -c -O3 -Wall -fPIC -o ./obj/funs.o ./src/funs.c -I./include/
!gcc -c -O3 -Wall -fPIC -o ./obj/statistics.o  ./src/statistics.c -I./include/
!gcc -shared -o ./bin/libfuns.dll  ./obj/funs.o  ./obj/statistics.o

In [21]:
!dir .\bin\libf*.dll

 Volume in drive F is cmh
 Volume Serial Number is 9C25-3306

 Directory of F:\SEU\SEE\PySEE\home\notebook\demo\bin

2020/03/12  12:14           127,040 libfuns.dll
               1 File(s)        127,040 bytes
               0 Dir(s)  98,837,725,184 bytes free


#### 3.2.2 Building with makefile under Windows


In [22]:
%%file ./makefile-libfuns.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: libfuns

libfuns: funsobj
	 $(CC) -shared -o $(BINDIR)libfuns.dll $(OBJDIR)funs.o $(OBJDIR)statistics.o
   
funsobj: $(SRCDIR)funs.c $(SRCDIR)statistics.c
	$(CC) -c $(CFLAGS) -o $(OBJDIR)statistics.o $(SRCDIR)statistics.c -I$(INCDIR)
	$(CC) -c $(CFLAGS) -o $(OBJDIR)funs.o $(SRCDIR)funs.c -I$(INCDIR)


Writing ./makefile-libfuns.mk


In [23]:
!make -f makefile-libfuns.mk

gcc -c -O3 -Wall -fPIC -o ./obj/statistics.o ./src/statistics.c -I./include/
gcc -c -O3 -Wall -fPIC -o ./obj/funs.o ./src/funs.c -I./include/
gcc -shared -o ./bin/libfuns.dll ./obj/funs.o ./obj/statistics.o


The result is a compiled shared library **`libfuns.dll`**

### 3.3 Building a client executable 

The following source code `"funsApp.c"` demonstrates calling the DLL's functions:


In [25]:
%%file ./src/funsApp.c

#include <stdio.h> 
#include "statistics.h"
#include "funs.h"

int main() {
    
     double a[] = {8, 4, 5, 3, 2};
     int len= sizeof(a)/sizeof(*a);
     printf("mena is %f\n", mean(a,  len)); 
    
     int n =5;
     printf("the factorial of  %d is %d\n",n,factorial(n));  // 5!=120
     return 0;
}

Overwriting ./src/funsApp.c


#### Windows 

In [26]:
!gcc -c -o ./obj/funsApp.o ./src/funsApp.c -I./include/
!gcc -o  ./bin/funsApp ./obj/funsApp.o  -L./bin/ -lfuns

In [27]:
!.\bin\funsApp

mena is 4.400000
the factorial of  5 is 120


#### Linux

In [ ]:
!gcc -c -o ./obj/funsApp.o ./src/funsApp.c -I./include/
!gcc -o  ./bin/funsApp ./obj/funsApp.o  -L./bin/ -lfuns -Wl,-rpath=./bin/ 

In [ ]:
!./bin/funsApp

## Reference

* GCC (GNU compilers) http://gcc.gnu.org

  * GCC Manual  http://gcc.gnu.org/onlinedocs

  * An Introduction to GCC  http://www.network-theory.co.uk/docs/gccintro/index.html.

  * GCC and Make：Compiling, Linking and Building C/C++ Applications http://www3.ntu.edu.sg/home/ehchua/programming/cpp/gcc_make.html

  * MinGW-W64 (GCC) Compiler Suite: http://www.mingw-w64.org/doku.php


* C/C++ for VS Code https://code.visualstudio.com/docs/languages/cpp

* C/C++ Preprocessor Directives http://www.cplusplus.com/doc/tutorial/preprocessor/


* What is a DLL and How Do I Create or Use One? http://www.mingw.org/wiki/DLL

